In [168]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from numpy.linalg import norm
import pandas as pd
import numpy as np
import requests
import re
import multiprocessing
import plotly.graph_objects as go
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer


## JD model

In [5]:
jd_data = pd.read_csv('data/jobs_data/JobsDataset.csv') 
jd_data.drop(['ID','Query'], axis=1, inplace=True)
jd_data['data'] = jd_data[['Job Title','Description']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
tagged_data = [TaggedDocument(words = word_tokenize(_d.lower()), tags = [str(i)]) for i, _d in enumerate(jd_data['data'])]

In [6]:
epochs_list = [80, 100, 150]

for epochs in epochs_list:
    model = Doc2Vec(vector_size = 50,
    min_count = 3,
    epochs = epochs,
    alpha = 0.001,
    workers = 16
    )
    model.build_vocab(tagged_data)
    model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
    
    # Save or use the trained model here based on your needs
    model.save(f'doc2vec_model_epochs_{epochs}.model')

In [24]:


jd_data['data'] = jd_data[['Job Title','Description']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
tagged_data = [TaggedDocument(words = word_tokenize(_d.lower()), tags = [str(i)]) for i, _d in enumerate(jd_data['data'])]
model = Doc2Vec(vector_size = 50,
min_count = 3,
epochs = 40,
alpha = 0.001,
workers = 4
)
model.build_vocab(tagged_data)

In [15]:
jd_data

,Job Title,Description,data
0,Junior Data Scientist Apprenticeship,Job Description As a Junior Data Scientist at ...,Junior Data Scientist Apprenticeship Job Descr...
1,"HBO Data Scientist, Content Science",OVERALL SUMMARY As a Data Scientist on the Dat...,"HBO Data Scientist, Content Science OVERALL SU..."
2,Junior Data Scientist,The Team: The Data science team is a newly for...,Junior Data Scientist The Team: The Data scie...
3,Jr Data Scientist,We now have a need for junior Data Scientist(s...,Jr Data Scientist We now have a need for junio...
4,"Data Scientist, Premium Content",Do you want to help guide the core business of...,"Data Scientist, Premium Content Do you want t..."
...,...,...,...
9993,Learning Architect,The opportunity Our customers understand that ...,Learning Architect The opportunity Our custome...
9994,Systems Architect,The NASA Ames Research Center has the requirem...,Systems Architect The NASA Ames Research Cente...
9995,Hardware Architect,What You'll Do Distinguished Engineer with pro...,Hardware Architect What You'll Do Distinguish...
9996,"Software Development Engineer, Full Stack – E...","Software Development Engineer, Full Stack – En...","Software Development Engineer, Full Stack – E..."


In [25]:
# Vocabulary building
model.build_vocab(tagged_data)
# Get the vocabulary keys
keys = model.wv.key_to_index.keys()
# Print the length of the vocabulary keys
print(len(keys))

27351


In [23]:
num_workers = multiprocessing.cpu_count()  # Use all available cores (4 in your case)
num_workers

16

In [26]:
# Train the model
for epoch in range(model.epochs):
    print(f"Training epoch {epoch+1}/{model.epochs}")
    model.train(tagged_data, 
                total_examples=model.corpus_count, 
                epochs=model.epochs)

model.save('cv_job_maching.model')
print("Model saved")

Training epoch 1/40
Training epoch 2/40
Training epoch 3/40
Training epoch 4/40
Training epoch 5/40
Training epoch 6/40
Training epoch 7/40
Training epoch 8/40
Training epoch 9/40
Training epoch 10/40
Training epoch 11/40
Training epoch 12/40
Training epoch 13/40
Training epoch 14/40
Training epoch 15/40
Training epoch 16/40
Training epoch 17/40
Training epoch 18/40
Training epoch 19/40
Training epoch 20/40
Training epoch 21/40
Training epoch 22/40
Training epoch 23/40
Training epoch 24/40
Training epoch 25/40
Training epoch 26/40
Training epoch 27/40
Training epoch 28/40
Training epoch 29/40
Training epoch 30/40
Training epoch 31/40
Training epoch 32/40
Training epoch 33/40
Training epoch 34/40
Training epoch 35/40
Training epoch 36/40
Training epoch 37/40
Training epoch 38/40
Training epoch 39/40
Training epoch 40/40
Model saved


In [28]:
def preprocess_text(text):
    # Convert the text to lowercase
    text = text.lower()
    
    # Remove punctuation from the text
    text = re.sub('[^a-z]', ' ', text)
    
    # Remove numerical values from the text
    text = re.sub(r'\d+', '', text)
    
    # Remove extra whitespaces
    text = ' '.join(text.split())
    
    return 
def remove_accents(input_string):
    accents = {'é': 'e', 'è': 'e', 'ê': 'e', 'ë': 'e', 'É': 'E', 'È': 'E', 'Ê': 'E', 'Ë': 'E',
        'à': 'a',
        'â': 'a',
        'ç': 'c',
        'î': 'i',
        'ï': 'i',
        'ô': 'o',
        'ù': 'u',
        'û': 'u'
        # Add more replacements as needed
    }
def remove_initial_newlines(text):
    sentence_found = False
    new_text = ''

    for char in text:
        if not sentence_found:
            if char == '\n':
                continue
            else:
                sentence_found = True
                new_text += char
        else:
            new_text += char

    return new_text
def remove_first_lines(text):
    first_lines_removed= remove_initial_newlines(text)
    return '\n'.join(first_lines_removed.split('\n', 1)[1:])

In [ ]:
import sys
sys.path.append('..')
from tika import parser
def read_file(file,path):
  """Reads a file and returns the content."""
  try:
    parsed_pdf = parser.from_file(path+'/'+file)
    non_accent = remove_accents(parsed_pdf['content'].lower())
    return remove_first_lines(non_accent) 
  except Exception as e:
    # Catch any other exceptions and log error message
    logging.error("Error while reading the file: " + str(e))
    return None


In [3]:
jd_data = pd.read_csv('data/jobs_data/JobsDataset.csv') 


In [240]:
scores = pd.read_csv('data/scores_db_data.csv') 
model_80 = Doc2Vec.load('doc2vec_model_epochs_80.model')
model_100 = Doc2Vec.load('doc2vec_model_epochs_100.model')
model_150 = Doc2Vec.load('doc2vec_model_epochs_150.model')

In [241]:
scores = pd.read_csv('data/scores_db_data.csv')

In [52]:
import pandas as pd

data = [
    ("model80", "react", "CV_Amine_Gafsi.pdf", 39.359495, 5.0),
    ("model100", "react", "CV_Amine_Gafsi.pdf", 32.219495, 6.0),
    ("model150", "react", "CV_Amine_Gafsi.pdf", 28.419495, 6.0),
    ("model80", "laravel", "CV_Amine_Gafsi.pdf", 34.438757, 5.0),
    ("model100", "laravel", "CV_Amine_Gafsi.pdf", 30.053757, 6.0),
    ("model150", "laravel", "CV_Amine_Gafsi.pdf", 23.308757, 7.0),
    ("model80", "react", "laravel.pdf", 49.989495, 4.0),
    ("model100", "react", "laravel.pdf", 49.459495, 4.0),
    ("model150", "react", "laravel.pdf", 47.399495, 4.0),
    ("model80", "laravel", "laravel.pdf", 52.281852, 3.0),
    ("model100", "laravel", "laravel.pdf", 48.561852, 4.0),
    ("model150", "laravel", "laravel.pdf", 48.486852, 4.0),
    ("model80", "react", "rayen.pdf", 27.104356, 7.0),
    ("model100", "react", "rayen.pdf", 25.744356, 7.0),
    ("model150", "react", "rayen.pdf", 27.759356, 7.0),
    ("model80", "laravel", "rayen.pdf", 30.883287, 7.0),
    ("model100", "laravel", "rayen.pdf", 28.218287, 7.0),
    ("model150", "laravel", "rayen.pdf", 34.018287, 5.0),
    ("model80", "react", "react3.pdf", 50.766428, 3.0),
    ("model100", "react", "react3.pdf", 52.576428, 2.0),
    ("model150", "react", "react3.pdf", 52.596428, 2.0),
    ("model80", "laravel", "react3.pdf", 49.413287, 4.0),
    ("model100", "laravel", "react3.pdf", 48.603287, 3.0),
    ("model150", "laravel", "react3.pdf", 50.028287, 3.0),
    ("model80", "react", "react_native.pdf", 57.985000, 1.0),
    ("model100", "react", "react_native.pdf", 55.725000, 1.0),
    ("model150", "react", "react_native.pdf", 53.400000, 1.0),
    ("model80", "laravel", "react_native.pdf", 53.378287, 2.0),
    ("model100", "laravel", "react_native.pdf", 52.938287, 2.0),
    ("model150", "laravel", "react_native.pdf", 54.128287, 1.0),
    ("model80", "react", "react_native2.pdf", 33.875000, 6.0),
    ("model100", "react", "react_native2.pdf", 32.980000, 5.0),
    ("model150", "react", "react_native2.pdf", 33.950000, 5.0),
    ("model80", "laravel", "react_native2.pdf", 32.195000, 6.0),
    ("model100", "laravel", "react_native2.pdf", 31.885000, 5.0),
    ("model150", "laravel", "react_native2.pdf", 32.870000, 6.0),
    ("model80", "react", "react_native4.pdf", 51.510000, 2.0),
    ("model100", "react", "react_native4.pdf", 50.410000, 3.0),
    ("model150", "react", "react_native4.pdf", 49.690000, 3.0),
    ("model80", "laravel", "react_native4.pdf", 56.816852, 1.0),
    ("model100", "laravel", "react_native4.pdf", 55.131852, 1.0),
    ("model150", "laravel", "react_native4.pdf", 53.146852, 2.0),
    ("model80", "react", "scrum_master.pdf", 13.745000, 8.0),
    ("model100", "react", "scrum_master.pdf", 9.945000, 8.0),
    ("model150", "react", "scrum_master.pdf", 14.270000, 8.0),
    ("model80", "laravel", "scrum_master.pdf", 24.010000, 8.0),
    ("model100", "laravel", "scrum_master.pdf", 17.885000, 8.0),
    ("model150", "laravel", "scrum_master.pdf", 21.940000, 8.0)
]

df_pred = pd.DataFrame(data, columns=["model", "type", "name", "similarity", "pred_ranking"])
df_pred


,model,type,name,similarity,pred_ranking
0,model80,react,CV_Amine_Gafsi.pdf,39.359495,5.0
1,model100,react,CV_Amine_Gafsi.pdf,32.219495,6.0
2,model150,react,CV_Amine_Gafsi.pdf,28.419495,6.0
3,model80,laravel,CV_Amine_Gafsi.pdf,34.438757,5.0
4,model100,laravel,CV_Amine_Gafsi.pdf,30.053757,6.0
5,model150,laravel,CV_Amine_Gafsi.pdf,23.308757,7.0
6,model80,react,laravel.pdf,49.989495,4.0
7,model100,react,laravel.pdf,49.459495,4.0
8,model150,react,laravel.pdf,47.399495,4.0
9,model80,laravel,laravel.pdf,52.281852,3.0


In [2]:
pred_results = pd.read_csv("perdicted_results.csv")

In [3]:
pred_results

,Unnamed: 0,model,type,name,similarity,ranking
0,0,model80,react,CV_Amine_Gafsi.pdf,39.689495,5.0
1,1,model100,react,CV_Amine_Gafsi.pdf,32.449495,6.0
2,2,model150,react,CV_Amine_Gafsi.pdf,28.574495,6.0
3,3,model40,react,CV_Amine_Gafsi.pdf,50.544495,6.0
4,4,model80,laravel,CV_Amine_Gafsi.pdf,34.563757,5.0
...,...,...,...,...,...,...
59,59,model40,react,scrum_master.pdf,31.185000,8.0
60,60,model80,laravel,scrum_master.pdf,24.130000,8.0
61,61,model100,laravel,scrum_master.pdf,17.875000,8.0
62,62,model150,laravel,scrum_master.pdf,21.960000,8.0


In [242]:
scores['name']

0                 CV_Hana_Zrafi
1        CV_Med_Houssem_Beltifa
2      CV_Mejdi_Belhadj_Youssef
3    CV_Mohamed_Fawzi_Stambouli
4       CV_Ons_Ben_Hadj_Sghaier
5        CV_Rayen_Ben_Fathallah
6                 CV_Wael_Antar
7            CV_Wissem_Boujlida
8              CV_Yasmine_Bizid
9                    rayen_data
Name: name, dtype: object

In [243]:
scores['name'] = scores['name']+'.pdf'


In [244]:
merged_df = pd.merge(pred_results, scores, on=["name", "type"])


NameError: name 'pred_results' is not defined

In [9]:
merged_df

,Unnamed: 0,model,type,name,similarity,ranking_x,ranking_y
0,0,model80,react,CV_Amine_Gafsi.pdf,39.689495,5.0,7
1,1,model100,react,CV_Amine_Gafsi.pdf,32.449495,6.0,7
2,2,model150,react,CV_Amine_Gafsi.pdf,28.574495,6.0,7
3,3,model40,react,CV_Amine_Gafsi.pdf,50.544495,6.0,7
4,4,model80,laravel,CV_Amine_Gafsi.pdf,34.563757,5.0,8
...,...,...,...,...,...,...,...
59,59,model40,react,scrum_master.pdf,31.185000,8.0,8
60,60,model80,laravel,scrum_master.pdf,24.130000,8.0,7
61,61,model100,laravel,scrum_master.pdf,17.875000,8.0,7
62,62,model150,laravel,scrum_master.pdf,21.960000,8.0,7


### Jd model evalutions

In [30]:
merged_df['ranking_x'] = merged_df['ranking_x'].astype('int64')


<class 'pandas.core.frame.DataFrame'>
Int64Index: 64 entries, 32 to 63
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  64 non-null     int64  
 1   model       64 non-null     object 
 2   type        64 non-null     object 
 3   name        64 non-null     object 
 4   similarity  64 non-null     float64
 5   ranking_x   64 non-null     int64  
 6   ranking_y   64 non-null     int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 4.0+ KB


In [54]:
merged_df.sort_values(by='ranking_x',ascending=True,inplace=True)
models=merged_df['model'].unique().tolist()

['model80', 'model100', 'model150', 'model40']

In [79]:
test_df=merged_df[['ranking_x','ranking_y']] <= 4
deno = test_df['ranking_x'].sum()
test_df

,ranking_x,ranking_y
32,True,True
33,True,True
34,True,True
35,True,True
52,True,False
...,...,...
61,False,False
62,False,False
60,False,False
59,False,False


In [81]:
deno

32

In [80]:
test_df['ranking_x'].value_counts()

True     32
False    32
Name: ranking_x, dtype: int64

In [83]:
def precision_at_k(df: pd.DataFrame, k: int=3, y_test: str='ranking_y', y_pred: str='ranking_x') -> float:
    if k <= 0:
        raise ValueError('Value of k should be greater than 1, read in as: {}'.format(k))
    # check y_test & y_pred columns are in df
    if y_test not in df.columns:
        raise ValueError('Input dataframe does not have a column named: {}'.format(y_test))
    if y_pred not in df.columns:
        raise ValueError('Input dataframe does not have a column named: {}'.format(y_pred))
    dfK = df.head(k)
    denominator = dfK[y_pred].sum()
    numerator = dfK[dfK[y_pred] & dfK[y_test]].shape[0]
    if denominator > 0:
        return numerator/denominator
    else:
        return None
#precision_at_k(df,3,'ranking_y','ranking_x') 

In [86]:
threshold = 3
for model in models : 
    model_df = merged_df[merged_df['model']==model]
    model_df=model_df[['ranking_x','ranking_y']] <= threshold
    precision = precision_at_k(model_df,3,'ranking_y','ranking_x')
    print(f'percision for the {model} is : {precision}')

percision for the model80 is : 0.6666666666666666
percision for the model100 is : 0.6666666666666666
percision for the model150 is : 1.0
percision for the model40 is : 1.0


In [68]:
from scipy.stats import kendalltau


In [74]:
models = ["model40","model80", "model100", "model150"]
kendall_tau_scores = {}

for model in models:
    model_data = merged_df[merged_df["model"] == model]
    kendall_tau, _ = kendalltau(model_data["ranking_x"], model_data["ranking_y"])
    kendall_tau_scores[model] = kendall_tau

In [76]:
kendall_tau_scores

{'model40': 0.4821428571428571,
 'model80': 0.3839285714285714,
 'model100': 0.49107142857142855,
 'model150': 0.5535714285714285}

### Resume model

In [3]:
resumes = pd.read_csv('data/raw_data/resumes.csv')

In [4]:
import re
def cleanResume(resumeText):
    resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) 
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
    return resumeText
    
resumes['cleaned_resume'] = resumes.Resume.apply(lambda x: cleanResume(x))

In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rayen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
nltk.download('wordnet')
nltk.download('punkt')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rayen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rayen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
def preprocess_resume_text(text, tag):
    text = cleanResume(text)
    ''' 
stemming Example:

Original: jumping, jumps, jumped
Stemmed: jump
lemmetizing Example:

Original: jumping, jumps, jumped
Lemmatized: jump
'''
    filtered_text = ' '.join([word for word in word_tokenize(text) if word.lower() not in stop_words])
    tokens = word_tokenize(filtered_text)
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return TaggedDocument(words=lemmatized_tokens, tags=[str(tag)])

tagged_data_res_2 = [preprocess_resume_text(_d, i) for i, _d in enumerate(resumes['cleaned_resume'])]


In [19]:
epochs_list = [80, 100, 150]

for epochs in epochs_list:
    model = Doc2Vec(vector_size = 50,
    min_count = 4,
    epochs = epochs,
    alpha = 0.001,
    workers = 16
    )
    model.build_vocab(tagged_data_res_2)
    model.train(tagged_data_res_2, total_examples=model.corpus_count, epochs=model.epochs)
    
    # Save or use the trained model here based on your needs
    model.save(f'resumes_model_epochs_lemmetized_{epochs}.model')

In [15]:
!pip install google_trans

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for google_trans: filename=google_trans-2.4.0-py3-none-any.whl size=9920 sha256=b29ac3f11474c0a69c8fd446bce11fe505f425a9ad1f8ee87115150a3e9ddb68
  Stored in directory: c:\users\rayen\appdata\local\pip\cache\wheels\8e\79\79\0b0c70e7b759bf219679c67d3564ad0b9af343d298c2fc87a1
Successfully built google_trans


In [12]:
from googletrans import Translator
from langdetect import detect
def detect_lang(resume_text) :
  lang = detect(resume_text)
  if lang == 'fr' :
    final_text=list()
    for i in range(0, len(text), 50000):
      translation = translator.translate(final_text[i:i+50000], dest='en')
      translations.append(translation)
    return ' '.join(final_text)
resume_example = pd.read_csv('data/raw_data/CV_Ahmed_Farhat.pdf')
#preprocessed_resume = preprocess_resume_text(resume_example)
detect_lang(resume_example)

ModuleNotFoundError: No module named 'googletrans'

## Resume + jd models evaluation 

In [39]:
final_df = pd.read_csv('perdicted_results_final.csv')


In [42]:
final_df.drop('Unnamed: 0',axis=1,inplace=True)

In [43]:
final_df

,jd_model,resume_model,type,name,similarity
0,model80,model80,react,CV_Amine_Gafsi.pdf,49.719495
1,model80,model100,react,CV_Amine_Gafsi.pdf,50.684495
2,model80,model150,react,CV_Amine_Gafsi.pdf,41.809495
3,model100,model80,react,CV_Amine_Gafsi.pdf,51.999495
4,model100,model100,react,CV_Amine_Gafsi.pdf,52.009495
...,...,...,...,...,...
187,model150,model100,laravel,scrum_master.pdf,41.010000
188,model150,model150,laravel,scrum_master.pdf,30.765000
189,model40,model80,laravel,scrum_master.pdf,29.425000
190,model40,model100,laravel,scrum_master.pdf,29.770000


In [56]:
final_df['pred_ranking']=final_df.groupby(['type','jd_model','resume_model'])['similarity'].rank(ascending=False)
final_df=final_df.drop('ranking',axis=1)
final_df

,jd_model,resume_model,type,name,similarity,pred_ranking
0,model80,model80,react,CV_Amine_Gafsi.pdf,49.719495,7.0
1,model80,model100,react,CV_Amine_Gafsi.pdf,50.684495,6.0
2,model80,model150,react,CV_Amine_Gafsi.pdf,41.809495,4.0
3,model100,model80,react,CV_Amine_Gafsi.pdf,51.999495,7.0
4,model100,model100,react,CV_Amine_Gafsi.pdf,52.009495,6.0
...,...,...,...,...,...,...
187,model150,model100,laravel,scrum_master.pdf,41.010000,8.0
188,model150,model150,laravel,scrum_master.pdf,30.765000,8.0
189,model40,model80,laravel,scrum_master.pdf,29.425000,8.0
190,model40,model100,laravel,scrum_master.pdf,29.770000,8.0


In [58]:
merged_df = pd.merge(final_df, scores, on=["name", "type"])
merged_df

,jd_model,resume_model,type,name,similarity,pred_ranking,ranking
0,model80,model80,react,CV_Amine_Gafsi.pdf,49.719495,7.0,6
1,model80,model100,react,CV_Amine_Gafsi.pdf,50.684495,6.0,6
2,model80,model150,react,CV_Amine_Gafsi.pdf,41.809495,4.0,6
3,model100,model80,react,CV_Amine_Gafsi.pdf,51.999495,7.0,6
4,model100,model100,react,CV_Amine_Gafsi.pdf,52.009495,6.0,6
...,...,...,...,...,...,...,...
187,model150,model100,laravel,scrum_master.pdf,41.010000,8.0,7
188,model150,model150,laravel,scrum_master.pdf,30.765000,8.0,7
189,model40,model80,laravel,scrum_master.pdf,29.425000,8.0,7
190,model40,model100,laravel,scrum_master.pdf,29.770000,8.0,7


In [59]:
merged_df['pred_ranking']=merged_df['pred_ranking'].astype('int64')
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192 entries, 0 to 191
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   jd_model      192 non-null    object 
 1   resume_model  192 non-null    object 
 2   type          192 non-null    object 
 3   name          192 non-null    object 
 4   similarity    192 non-null    float64
 5   pred_ranking  192 non-null    int64  
 6   ranking       192 non-null    int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 12.0+ KB


In [69]:
jd_models = merged_df['jd_model'].unique()
resume_models= merged_df['resume_model'].unique()

In [71]:
jd_models

array(['model80', 'model100', 'model150', 'model40'], dtype=object)

In [72]:
resume_models

array(['model80', 'model100', 'model150'], dtype=object)

In [91]:
mask = (merged_df['jd_model']=='model100') & (merged_df['resume_model'] == 'model100') 
model_df = merged_df[mask]
model_df.sort_values(by='pred_ranking',ascending=True,inplace=True)
model_df

C:\Users\rayen\AppData\Local\Temp\ipykernel_3424\3509835854.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df.sort_values(by='pred_ranking',ascending=True,inplace=True)


,jd_model,resume_model,type,name,similarity,pred_ranking,ranking
76,model100,model100,react,react3.pdf,61.761428,1,1
88,model100,model100,laravel,react3.pdf,61.803287,1,6
64,model100,model100,laravel,rayen.pdf,61.583287,2,3
100,model100,model100,react,react_native.pdf,58.940000,2,3
112,model100,model100,laravel,react_native.pdf,59.813287,3,2
148,model100,model100,react,react_native4.pdf,54.570000,3,2
28,model100,model100,react,laravel.pdf,52.689495,4,5
40,model100,model100,laravel,laravel.pdf,58.051852,4,1
16,model100,model100,laravel,CV_Amine_Gafsi.pdf,55.378757,5,8
124,model100,model100,react,react_native2.pdf,52.425000,5,4


In [98]:
threshold = 3
merged_df.sort_values(by='pred_ranking',ascending=True,inplace=True)
for jd_model in jd_models :
    for resume_model in resume_models : 
        mask = (merged_df['jd_model']==jd_model) & (merged_df['resume_model'] == resume_model) 
        model_df = merged_df[mask]
        model_df=model_df[['pred_ranking','ranking']] <= threshold
        precision = precision_at_k(model_df,3,'ranking','pred_ranking')
        print(f'percision for the {jd_model} and {resume_model} is : {precision}')
        


percision for the model80 and model80 is : 0.6666666666666666
percision for the model80 and model100 is : 0.6666666666666666
percision for the model80 and model150 is : 0.6666666666666666
percision for the model100 and model80 is : 0.6666666666666666
percision for the model100 and model100 is : 0.6666666666666666
percision for the model100 and model150 is : 1.0
percision for the model150 and model80 is : 0.6666666666666666
percision for the model150 and model100 is : 1.0
percision for the model150 and model150 is : 1.0
percision for the model40 and model80 is : 0.6666666666666666
percision for the model40 and model100 is : 0.3333333333333333
percision for the model40 and model150 is : 1.0


### Evaluation on larger dataset

### Test on data science jd 

In [428]:
final_scores = pd.read_csv('perdicted_results_final3.csv')

In [429]:
real_scores = pd.read_csv('data/scores_db_data.csv')

In [430]:
real_scores

,name,type,ranking
0,CV_Hana_Zrafi,data,8
1,CV_Med_Houssem_Beltifa,data,4
2,CV_Mejdi_Belhadj_Youssef,data,3
3,CV_Mohamed_Fawzi_Stambouli,data,9
4,CV_Ons_Ben_Hadj_Sghaier,data,10
5,CV_Rayen_Ben_Fathallah,data,2
6,CV_Wael_Antar,data,7
7,CV_Wissem_Boujlida,data,6
8,CV_Yasmine_Bizid,data,5
9,rayen_data,data,1


In [431]:
final_scores

,Unnamed: 0,jd_model,resume_model,type,name,similarity
0,0,model80,model80,react,CV_Ahlem_Marzougui.pdf,50.100406
1,1,model80,model100,react,CV_Ahlem_Marzougui.pdf,47.460406
2,2,model80,model150,react,CV_Ahlem_Marzougui.pdf,31.544406
3,3,model80,model80jd,react,CV_Ahlem_Marzougui.pdf,8.356406
4,4,model80,model100jd,react,CV_Ahlem_Marzougui.pdf,3.820406
...,...,...,...,...,...,...
6561,6561,model40jd,model150,data,yessine Meddeb_cv.pdf,19.887356
6562,6562,model40jd,model80jd,data,yessine Meddeb_cv.pdf,17.691356
6563,6563,model40jd,model100jd,data,yessine Meddeb_cv.pdf,15.683356
6564,6564,model40jd,model150jd,data,yessine Meddeb_cv.pdf,9.539356


In [432]:
final_scores = final_scores[final_scores['type']!='react']

In [433]:
final_scores

,Unnamed: 0,jd_model,resume_model,type,name,similarity
49,49,model80,model80,data,CV_Ahlem_Marzougui.pdf,43.767356
50,50,model80,model100,data,CV_Ahlem_Marzougui.pdf,42.891356
51,51,model80,model150,data,CV_Ahlem_Marzougui.pdf,28.743356
52,52,model80,model80jd,data,CV_Ahlem_Marzougui.pdf,5.903356
53,53,model80,model100jd,data,CV_Ahlem_Marzougui.pdf,4.815356
...,...,...,...,...,...,...
6561,6561,model40jd,model150,data,yessine Meddeb_cv.pdf,19.887356
6562,6562,model40jd,model80jd,data,yessine Meddeb_cv.pdf,17.691356
6563,6563,model40jd,model100jd,data,yessine Meddeb_cv.pdf,15.683356
6564,6564,model40jd,model150jd,data,yessine Meddeb_cv.pdf,9.539356


In [434]:
real_scores['name']= real_scores['name']+'.pdf'

In [435]:
pred_names=list(final_scores['name'].unique()) 

In [436]:
real_names = list(real_scores['name'].unique())
pred_names=list(final_scores['name'].unique()) 
difference = list(set(pred_names) - set(real_names))

In [437]:
difference = list(set(pred_names) - set(real_names))

In [438]:
len(real_names) 

10

In [439]:
len(difference) 

57

In [440]:
real_scores

,name,type,ranking
0,CV_Hana_Zrafi.pdf,data,8
1,CV_Med_Houssem_Beltifa.pdf,data,4
2,CV_Mejdi_Belhadj_Youssef.pdf,data,3
3,CV_Mohamed_Fawzi_Stambouli.pdf,data,9
4,CV_Ons_Ben_Hadj_Sghaier.pdf,data,10
5,CV_Rayen_Ben_Fathallah.pdf,data,2
6,CV_Wael_Antar.pdf,data,7
7,CV_Wissem_Boujlida.pdf,data,6
8,CV_Yasmine_Bizid.pdf,data,5
9,rayen_data.pdf,data,1


In [441]:
difference

['scrum_master.pdf',
 'CV_Eya_Rejeb.pdf',
 'CV_Yessine_Miled.pdf',
 'CV_Aya_Benjazia.pdf',
 'CV_khawla_belkacem.pdf',
 'CV_Ahmed_Dziri.pdf',
 'CV_Ben_Chaaben_Yassin.pdf',
 'CV_Iheb_Alouane.pdf',
 'CV_Anwer_Boubaker.pdf',
 'CV_Ahlem_Marzougui.pdf',
 'CV_Sarra_Laribi.pdf',
 'yessine Meddeb_cv.pdf',
 'CV_Zaddem_Raslèn.pdf',
 'CV_Saber_ben slama.pdf',
 'CV_Ayoub_Ben_Latifa.pdf',
 'CV_Ahmed_Farhat.pdf',
 'CV_Chedy_Sakly.pdf',
 'react_native.pdf',
 'CV_Rim_Thabet.pdf',
 'CV_Fadi_Zaafrane.pdf',
 'CV_Rania_Slimene.pdf',
 'experience.pdf',
 'CV_Ahmed_Amamou.pdf',
 'CV_Sanda_Dhouib.pdf',
 'CV_Med_Neji_Benzarti.pdf',
 'CV_Houssem_Kraiem.pdf',
 'CV_Oumeima_Zaddem.pdf',
 'CV_Ibrahim_Youssef.pdf',
 'resumes.csv',
 'CV_Amine_Gafsi.pdf',
 'CV_EYA_Sekma.pdf',
 'CV_Aymen_Ghnia.pdf',
 'CV_Rima_Jerbi.pdf',
 'CV_Souheyl_Gouadria.pdf',
 'CV_Med_Oussema_Zaier.pdf',
 'CV_Houssem_Hassani.pdf',
 'CV_Ibtihel_Khdhaouria.pdf',
 'CV_Khaled_Chtioui.pdf',
 'CV_Oussema_Mili.pdf',
 'CV_Mihed_Hassine.pdf',
 'CV_Med_Nour

In [442]:
new_data = {'name': difference,
            'type': 'data',
            'ranking': 30}
new_df = pd.DataFrame(new_data)
final_df = pd.concat([real_scores, new_df], ignore_index=True)

In [443]:
difference2 = list(set(real_names) - set(pred_names))


In [444]:
final_df = final_df[~final_df['name'].isin(difference2)]


In [445]:
final_df

,name,type,ranking
0,CV_Hana_Zrafi.pdf,data,8
1,CV_Med_Houssem_Beltifa.pdf,data,4
2,CV_Mejdi_Belhadj_Youssef.pdf,data,3
3,CV_Mohamed_Fawzi_Stambouli.pdf,data,9
4,CV_Ons_Ben_Hadj_Sghaier.pdf,data,10
...,...,...,...
62,CV_Lina_Ounaies.pdf,data,30
63,Dziri Ahmed - Resume (1).pdf,data,30
64,CV_Yassine_Afsa.pdf,data,30
65,CV_Zeineb_Bachouch.pdf,data,30


In [446]:
final_scores['pred_ranking']=final_scores.groupby(['type','jd_model','resume_model'])['similarity'].rank(ascending=False)


C:\Users\rayen\AppData\Local\Temp\ipykernel_8236\3698117208.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_scores['pred_ranking']=final_scores.groupby(['type','jd_model','resume_model'])['similarity'].rank(ascending=False)


In [447]:
final_df=final_df.rename(columns={"name": "name"}) 

In [448]:
merged_df = pd.merge(final_df, final_scores, on=["name", "type"])
merged_df

,name,type,ranking,Unnamed: 0,jd_model,resume_model,similarity,pred_ranking
0,CV_Hana_Zrafi.pdf,data,8,1813,model80,model80,45.769356,26.0
1,CV_Hana_Zrafi.pdf,data,8,1814,model80,model100,46.185356,23.0
2,CV_Hana_Zrafi.pdf,data,8,1815,model80,model150,37.517356,30.0
3,CV_Hana_Zrafi.pdf,data,8,1816,model80,model80jd,1.909356,54.0
4,CV_Hana_Zrafi.pdf,data,8,1817,model80,model100jd,-0.126644,56.0
...,...,...,...,...,...,...,...,...
3278,CV_Nour_Mabrouk.pdf,data,30,3523,model40jd,model150,8.392577,56.0
3279,CV_Nour_Mabrouk.pdf,data,30,3524,model40jd,model80jd,18.444577,31.0
3280,CV_Nour_Mabrouk.pdf,data,30,3525,model40jd,model100jd,16.704577,30.0
3281,CV_Nour_Mabrouk.pdf,data,30,3526,model40jd,model150jd,7.952577,42.0


In [449]:
react_data = final_scores[final_scores['type']=='data']


In [450]:
react_data

,Unnamed: 0,jd_model,resume_model,type,name,similarity,pred_ranking
49,49,model80,model80,data,CV_Ahlem_Marzougui.pdf,43.767356,39.0
50,50,model80,model100,data,CV_Ahlem_Marzougui.pdf,42.891356,36.0
51,51,model80,model150,data,CV_Ahlem_Marzougui.pdf,28.743356,43.0
52,52,model80,model80jd,data,CV_Ahlem_Marzougui.pdf,5.903356,42.0
53,53,model80,model100jd,data,CV_Ahlem_Marzougui.pdf,4.815356,39.0
...,...,...,...,...,...,...,...
6561,6561,model40jd,model150,data,yessine Meddeb_cv.pdf,19.887356,24.0
6562,6562,model40jd,model80jd,data,yessine Meddeb_cv.pdf,17.691356,38.0
6563,6563,model40jd,model100jd,data,yessine Meddeb_cv.pdf,15.683356,38.0
6564,6564,model40jd,model150jd,data,yessine Meddeb_cv.pdf,9.539356,34.0


In [453]:
merged_df

,name,type,ranking,Unnamed: 0,jd_model,resume_model,similarity,pred_ranking
0,CV_Hana_Zrafi.pdf,data,8,1813,model80,model80,45.769356,26.0
1,CV_Hana_Zrafi.pdf,data,8,1814,model80,model100,46.185356,23.0
2,CV_Hana_Zrafi.pdf,data,8,1815,model80,model150,37.517356,30.0
3,CV_Hana_Zrafi.pdf,data,8,1816,model80,model80jd,1.909356,54.0
4,CV_Hana_Zrafi.pdf,data,8,1817,model80,model100jd,-0.126644,56.0
...,...,...,...,...,...,...,...,...
3278,CV_Nour_Mabrouk.pdf,data,30,3523,model40jd,model150,8.392577,56.0
3279,CV_Nour_Mabrouk.pdf,data,30,3524,model40jd,model80jd,18.444577,31.0
3280,CV_Nour_Mabrouk.pdf,data,30,3525,model40jd,model100jd,16.704577,30.0
3281,CV_Nour_Mabrouk.pdf,data,30,3526,model40jd,model150jd,7.952577,42.0


In [452]:
difference_df = pd.concat([merged_df.drop('ranking',axis=1), react_data]).drop_duplicates(keep=False)


,name,type,Unnamed: 0,jd_model,resume_model,similarity,pred_ranking
0,CV_Hana_Zrafi.pdf,data,1813,model80,model80,45.769356,26.0
1,CV_Hana_Zrafi.pdf,data,1814,model80,model100,46.185356,23.0
2,CV_Hana_Zrafi.pdf,data,1815,model80,model150,37.517356,30.0
3,CV_Hana_Zrafi.pdf,data,1816,model80,model80jd,1.909356,54.0
4,CV_Hana_Zrafi.pdf,data,1817,model80,model100jd,-0.126644,56.0
...,...,...,...,...,...,...,...
3278,CV_Nour_Mabrouk.pdf,data,3523,model40jd,model150,8.392577,56.0
3279,CV_Nour_Mabrouk.pdf,data,3524,model40jd,model80jd,18.444577,31.0
3280,CV_Nour_Mabrouk.pdf,data,3525,model40jd,model100jd,16.704577,30.0
3281,CV_Nour_Mabrouk.pdf,data,3526,model40jd,model150jd,7.952577,42.0


In [454]:
merged_df['pred_ranking']=merged_df['pred_ranking'].astype('int64')

In [455]:
merged_df

,name,type,ranking,Unnamed: 0,jd_model,resume_model,similarity,pred_ranking
0,CV_Hana_Zrafi.pdf,data,8,1813,model80,model80,45.769356,26
1,CV_Hana_Zrafi.pdf,data,8,1814,model80,model100,46.185356,23
2,CV_Hana_Zrafi.pdf,data,8,1815,model80,model150,37.517356,30
3,CV_Hana_Zrafi.pdf,data,8,1816,model80,model80jd,1.909356,54
4,CV_Hana_Zrafi.pdf,data,8,1817,model80,model100jd,-0.126644,56
...,...,...,...,...,...,...,...,...
3278,CV_Nour_Mabrouk.pdf,data,30,3523,model40jd,model150,8.392577,56
3279,CV_Nour_Mabrouk.pdf,data,30,3524,model40jd,model80jd,18.444577,31
3280,CV_Nour_Mabrouk.pdf,data,30,3525,model40jd,model100jd,16.704577,30
3281,CV_Nour_Mabrouk.pdf,data,30,3526,model40jd,model150jd,7.952577,42


In [456]:
jd_models = merged_df['jd_model'].unique()
resume_models= merged_df['resume_model'].unique()

In [457]:
resume_models

array(['model80', 'model100', 'model150', 'model80jd', 'model100jd',
       'model150jd', 'model40jd'], dtype=object)

In [460]:
threshold = 20
precisions_data = []
merged_df.sort_values(by='pred_ranking',ascending=True,inplace=True)
for jd_model in jd_models :
    for resume_model in resume_models : 
        mask = (merged_df['jd_model']==jd_model) & (merged_df['resume_model'] == resume_model) 
        model_df = merged_df[mask]
        model_df=model_df[['pred_ranking','ranking']] <= threshold
        precision = precision_at_k(model_df,10,'ranking','pred_ranking')
        print(f'percision for the {jd_model} and {resume_model} is : {precision}')
        precisions_data.append({'jd_model' : jd_model,'resume_model' : resume_model,'precision' :precision})



percision for the model80 and model80 is : 0.8
percision for the model80 and model100 is : 0.8
percision for the model80 and model150 is : 0.6
percision for the model80 and model80jd is : 0.4
percision for the model80 and model100jd is : 0.3
percision for the model80 and model150jd is : 0.3
percision for the model80 and model40jd is : 0.4
percision for the model100 and model80 is : 0.7
percision for the model100 and model100 is : 0.7
percision for the model100 and model150 is : 0.6
percision for the model100 and model80jd is : 0.3
percision for the model100 and model100jd is : 0.2
percision for the model100 and model150jd is : 0.2
percision for the model100 and model40jd is : 0.4
percision for the model150 and model80 is : 0.7
percision for the model150 and model100 is : 0.6
percision for the model150 and model150 is : 0.4
percision for the model150 and model80jd is : 0.2
percision for the model150 and model100jd is : 0.3
percision for the model150 and model150jd is : 0.3
percision for

In [461]:
precisions_data

[{'jd_model': 'model80', 'resume_model': 'model80', 'precision': 0.8},
 {'jd_model': 'model80', 'resume_model': 'model100', 'precision': 0.8},
 {'jd_model': 'model80', 'resume_model': 'model150', 'precision': 0.6},
 {'jd_model': 'model80', 'resume_model': 'model80jd', 'precision': 0.4},
 {'jd_model': 'model80', 'resume_model': 'model100jd', 'precision': 0.3},
 {'jd_model': 'model80', 'resume_model': 'model150jd', 'precision': 0.3},
 {'jd_model': 'model80', 'resume_model': 'model40jd', 'precision': 0.4},
 {'jd_model': 'model100', 'resume_model': 'model80', 'precision': 0.7},
 {'jd_model': 'model100', 'resume_model': 'model100', 'precision': 0.7},
 {'jd_model': 'model100', 'resume_model': 'model150', 'precision': 0.6},
 {'jd_model': 'model100', 'resume_model': 'model80jd', 'precision': 0.3},
 {'jd_model': 'model100', 'resume_model': 'model100jd', 'precision': 0.2},
 {'jd_model': 'model100', 'resume_model': 'model150jd', 'precision': 0.2},
 {'jd_model': 'model100', 'resume_model': 'model4

### Test on react jd 

In [323]:
final_scores_react = pd.read_csv('perdicted_results_final3.csv')
real_scores_react = pd.read_csv('data/scores_db.csv')

In [327]:
final_scores_react.drop('Unnamed: 0',axis=1,inplace=True)

In [329]:
real_scores_react

,name,type,ranking
0,CV_Amine_Gafsi,react,5
1,CV_Anwer_Boubaker,react,7
2,CV_Aymen_Ghnia,react,3
3,CV_Ayoub_Ben_Latifa,react,40
4,CV_Fatma_Bouzgarrou,react,11
5,CV_Houssem_Hassani,react,30
6,CV_Ibtihel_Khdhaouria,react,6
7,CV_khawla_belkacem,react,13
8,CV_Lina_Ounaies,react,50
9,CV_Med_Amine_Mansour,react,51


In [333]:
final_scores_react = final_scores_react[final_scores_react['type']=='react']
final_scores_react

,jd_model,resume_model,type,name,similarity
0,model80,model80,react,CV_Ahlem_Marzougui.pdf,50.100406
1,model80,model100,react,CV_Ahlem_Marzougui.pdf,47.460406
2,model80,model150,react,CV_Ahlem_Marzougui.pdf,31.544406
3,model80,model80jd,react,CV_Ahlem_Marzougui.pdf,8.356406
4,model80,model100jd,react,CV_Ahlem_Marzougui.pdf,3.820406
...,...,...,...,...,...
6512,model40jd,model150,react,yessine Meddeb_cv.pdf,21.577708
6513,model40jd,model80jd,react,yessine Meddeb_cv.pdf,12.505708
6514,model40jd,model100jd,react,yessine Meddeb_cv.pdf,10.705708
6515,model40jd,model150jd,react,yessine Meddeb_cv.pdf,7.717708


In [363]:
real_scores_react=real_scores_react.rename(columns={"name ": "name"}) 
real_scores_react['name']=real_scores_react['name']+'.pdf'
real_scores_react

,name,type,ranking
0,CV_Amine_Gafsi.pdf,react,5
1,CV_Anwer_Boubaker.pdf,react,7
2,CV_Aymen_Ghnia.pdf,react,3
3,CV_Ayoub_Ben_Latifa.pdf,react,40
4,CV_Fatma_Bouzgarrou.pdf,react,11
5,CV_Houssem_Hassani.pdf,react,30
6,CV_Ibtihel_Khdhaouria.pdf,react,6
7,CV_khawla_belkacem.pdf,react,13
8,CV_Lina_Ounaies.pdf,react,50
9,CV_Med_Amine_Mansour.pdf,react,51


In [373]:
real_names_react = list(real_scores_react['name'].unique())
pred_names_react=list(final_scores_react['name'].unique()) 
difference_react = list(set(pred_names_react) - set(real_names_react))

In [390]:
diff2 = set(real_names_react) - set(pred_names_react)

In [396]:
real_scores_react = real_scores_react[~real_scores_react['name'].isin(diff2)]

In [398]:
real_names_react = list(real_scores_react['name'].unique())
pred_names_react=list(final_scores_react['name'].unique()) 
difference_react = list(set(pred_names_react) - set(real_names_react))

In [403]:
new_data_react = {'name': difference_react,
            'type': 'react',
            'ranking': 30}
new_df_react = pd.DataFrame(new_data_react)
final_df_react = pd.concat([real_scores_react, new_df_react], ignore_index=True)

In [404]:
final_df_react

,name,type,ranking
0,CV_Amine_Gafsi.pdf,react,5
1,CV_Anwer_Boubaker.pdf,react,7
2,CV_Aymen_Ghnia.pdf,react,3
3,CV_Ayoub_Ben_Latifa.pdf,react,40
4,CV_Fatma_Bouzgarrou.pdf,react,11
...,...,...,...
63,CV_Hana_Zrafi.pdf,react,30
64,CV_Wael_Antar.pdf,react,30
65,CV_Zeineb_Bachouch.pdf,react,30
66,CV_Med_Houssem_Beltifa.pdf,react,30


In [415]:
final_scores_react

,jd_model,resume_model,type,name,similarity
0,model80,model80,react,CV_Ahlem_Marzougui.pdf,50.100406
1,model80,model100,react,CV_Ahlem_Marzougui.pdf,47.460406
2,model80,model150,react,CV_Ahlem_Marzougui.pdf,31.544406
3,model80,model80jd,react,CV_Ahlem_Marzougui.pdf,8.356406
4,model80,model100jd,react,CV_Ahlem_Marzougui.pdf,3.820406
...,...,...,...,...,...
6512,model40jd,model150,react,yessine Meddeb_cv.pdf,21.577708
6513,model40jd,model80jd,react,yessine Meddeb_cv.pdf,12.505708
6514,model40jd,model100jd,react,yessine Meddeb_cv.pdf,10.705708
6515,model40jd,model150jd,react,yessine Meddeb_cv.pdf,7.717708


In [416]:
final_scores_react['pred_ranking']=final_scores_react.groupby(['type','jd_model','resume_model'])['similarity'].rank(ascending=False)


C:\Users\rayen\AppData\Local\Temp\ipykernel_8236\2420376905.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_scores_react['pred_ranking']=final_scores_react.groupby(['type','jd_model','resume_model'])['similarity'].rank(ascending=False)


In [418]:
final_scores_react['pred_ranking']= final_scores_react['pred_ranking'].astype('int64')

C:\Users\rayen\AppData\Local\Temp\ipykernel_8236\2146964048.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_scores_react['pred_ranking']= final_scores_react['pred_ranking'].astype('int64')


In [419]:
final_scores_react

,jd_model,resume_model,type,name,similarity,pred_ranking
0,model80,model80,react,CV_Ahlem_Marzougui.pdf,50.100406,11
1,model80,model100,react,CV_Ahlem_Marzougui.pdf,47.460406,16
2,model80,model150,react,CV_Ahlem_Marzougui.pdf,31.544406,39
3,model80,model80jd,react,CV_Ahlem_Marzougui.pdf,8.356406,9
4,model80,model100jd,react,CV_Ahlem_Marzougui.pdf,3.820406,10
...,...,...,...,...,...,...
6512,model40jd,model150,react,yessine Meddeb_cv.pdf,21.577708,8
6513,model40jd,model80jd,react,yessine Meddeb_cv.pdf,12.505708,16
6514,model40jd,model100jd,react,yessine Meddeb_cv.pdf,10.705708,16
6515,model40jd,model150jd,react,yessine Meddeb_cv.pdf,7.717708,16


In [424]:
merged_df_react = pd.merge(final_df_react, final_scores_react, on=["name", "type"])
merged_df_react

,name,type,ranking,jd_model,resume_model,similarity,pred_ranking
0,CV_Amine_Gafsi.pdf,react,5,model80,model80,49.477708,14
1,CV_Amine_Gafsi.pdf,react,5,model80,model100,46.509708,19
2,CV_Amine_Gafsi.pdf,react,5,model80,model150,40.077708,12
3,CV_Amine_Gafsi.pdf,react,5,model80,model80jd,6.857708,14
4,CV_Amine_Gafsi.pdf,react,5,model80,model100jd,2.617708,14
...,...,...,...,...,...,...,...
3327,CV_Nour_Mabrouk.pdf,react,30,model40jd,model150,1.656000,63
3328,CV_Nour_Mabrouk.pdf,react,30,model40jd,model80jd,6.300000,48
3329,CV_Nour_Mabrouk.pdf,react,30,model40jd,model100jd,3.584000,53
3330,CV_Nour_Mabrouk.pdf,react,30,model40jd,model150jd,-1.140000,54


In [421]:
jd_models = merged_df_react['jd_model'].unique()
resume_models= merged_df_react['resume_model'].unique()

In [459]:
threshold = 20
precisions = []
merged_df_react.sort_values(by='pred_ranking',ascending=True,inplace=True)
for jd_model in jd_models :
    for resume_model in resume_models : 
        mask = (merged_df_react['jd_model']==jd_model) & (merged_df_react['resume_model'] == resume_model) 
        model_df = merged_df_react[mask]
        model_df=model_df[['pred_ranking','ranking']] <= threshold
        precision = precision_at_k(model_df,10,'ranking','pred_ranking')
        print(f'percision for the {jd_model} and {resume_model} is : {precision}')
        precisions.append({'jd_model' : jd_model,'resume_model' : resume_model,'precision' :precision})
        
        


percision for the model80 and model80 is : 0.3
percision for the model80 and model100 is : 0.3
percision for the model80 and model150 is : 0.1
percision for the model80 and model80jd is : 0.3
percision for the model80 and model100jd is : 0.3
percision for the model80 and model150jd is : 0.4
percision for the model80 and model40jd is : 0.3
percision for the model100 and model80 is : 0.3
percision for the model100 and model100 is : 0.3
percision for the model100 and model150 is : 0.2
percision for the model100 and model80jd is : 0.3
percision for the model100 and model100jd is : 0.3
percision for the model100 and model150jd is : 0.4
percision for the model100 and model40jd is : 0.3
percision for the model150 and model80 is : 0.5
percision for the model150 and model100 is : 0.4
percision for the model150 and model150 is : 0.3
percision for the model150 and model80jd is : 0.4
percision for the model150 and model100jd is : 0.4
percision for the model150 and model150jd is : 0.4
percision for

In [469]:
merged_dict = {}
for item in precisions + precisions_data:
    key = (item['jd_model'], item['resume_model'])
    
    if key not in merged_dict:
        merged_dict[key] = {'jd_model': item['jd_model'], 'resume_model': item['resume_model']}
        
    if 'precision1' not in merged_dict[key]:
        merged_dict[key]['precision1'] = item['precision']
    else:
        merged_dict[key]['precision2'] = item['precision']

# Calculate average precision and add to the dictionary
for key, values in merged_dict.items():
    average_precision = (values.get('precision1', 0) + values.get('precision2', 0)) / 2
    values['average_precision'] = average_precision
output = list(merged_dict.values())
output

[{'jd_model': 'model80',
  'resume_model': 'model80',
  'precision1': 0.3,
  'precision2': 0.8,
  'average_precision': 0.55},
 {'jd_model': 'model80',
  'resume_model': 'model100',
  'precision1': 0.3,
  'precision2': 0.8,
  'average_precision': 0.55},
 {'jd_model': 'model80',
  'resume_model': 'model150',
  'precision1': 0.1,
  'precision2': 0.6,
  'average_precision': 0.35},
 {'jd_model': 'model80',
  'resume_model': 'model80jd',
  'precision1': 0.3,
  'precision2': 0.4,
  'average_precision': 0.35},
 {'jd_model': 'model80',
  'resume_model': 'model100jd',
  'precision1': 0.3,
  'precision2': 0.3,
  'average_precision': 0.3},
 {'jd_model': 'model80',
  'resume_model': 'model150jd',
  'precision1': 0.4,
  'precision2': 0.3,
  'average_precision': 0.35},
 {'jd_model': 'model80',
  'resume_model': 'model40jd',
  'precision1': 0.3,
  'precision2': 0.4,
  'average_precision': 0.35},
 {'jd_model': 'model100',
  'resume_model': 'model80',
  'precision1': 0.3,
  'precision2': 0.7,
  'average